In [ ]:
import os
import gffutils
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Alphabet import IUPAC
import json
from pyfaidx import Fasta
from Bio import SeqIO
import re
from pprint import pprint as pp

In [ ]:
# globals
template_species_list = ["Bcur", "Bdor", "Bole", "Ccap"]
transvestigated_species_set = {'Bcor', 'Blat', 'Bzon', 'Afra', 'Bmin', 'Bjar', 'Aobl'}
gff_path = "./input/gff/"
fasta_path = "./input/fasta/"
groups_fn = "./input/groups_filtered_6181genes.txt"
output_path = "./output/"
intermediate_path = "./intermediate/"
aligned_fasta_path = "./input/aligned_13spp_fasta/"

# gap filter parameters
max_gap_percent = 0
max_gap_length = 0
# cds length filter parameters
min_cds_length = 400
max_cds_length = 600

In [ ]:
# create handles for all .db files in intermediate directory
gff_fn = {name.split('.gff.db')[0]: intermediate_path + name for name in os.listdir(intermediate_path) if ".gff.db" in name}
gff = {key: gffutils.FeatureDB(value) for key, value in gff_fn.items()}

# create handles for all .fasta files in fasta directory
fasta_fn = {name.split('.nt.fasta')[0]: fasta_path + name for name in os.listdir(fasta_path) if
         ((".nt.fasta" in name) and (".nt.fasta.fai" not in name))}
fasta = {key: Fasta(value) for key, value in fasta_fn.items()}
        
# import ortholog groups
with open(intermediate_path + "groups.json", 'r') as f:
    parent_groups = json.load(f)

# create handles for all .fasta files in aligned_13spp_fasta directory
aligned_fasta_fn = {name.split('.13spp')[0]: aligned_fasta_path + name for name in os.listdir(aligned_fasta_path) if
         ((".fasta.aln" in name) and (".fasta.aln.fai" not in name))}

In [ ]:
# define functions to parse coordinates of cds's from concatinated aligned fasta w/ n's and -'s
nnn = 50
def findBreakpoints(seq):
    breakpoints = []
    loc = 0
    regex = re.compile(r"n+[-+n+]*")
    while(True):
        #loc = seq.find(nnn, loc)
        match = regex.search(seq, loc)
        if not match:
            break
        if len(match.group().replace('-', '')) >= nnn:
            breakpoints.append(match.span())
        loc = match.end()
    return(breakpoints)

def findExonCoords(seq):
    breakpoints = findBreakpoints(seq)
    length = len(seq)

    if len(breakpoints) == 0:
        return([(0, length)])

    if len(breakpoints) == 1:
        bp = breakpoints[0]
        return([(0, bp[0]), (bp[1], length)])

    elif len(breakpoints) > 0:
        exonCoords = []
        exonCoords.append((0, breakpoints[0][0])) # first exon

        for i in range(len(breakpoints) + 1)[1:-1]: # all intermediate exons
            ex_start = breakpoints[i-1][1]
            ex_end = breakpoints[i][0]
            exonCoords.append((ex_start, ex_end))

        exonCoords.append((breakpoints[-1][1], length)) # last exon
        return(exonCoords)
    
def gapPercent(seq):
    seq = str(seq)
    gappedLen = len(seq)
    gapCount = seq.count('-')
    return( (100.0*gapCount)/gappedLen )

def longestGap(seq):
    seq = str(seq)
    gap_regex = re.compile(r"-+")
    gap_list = gap_regex.findall(seq)
    if gap_list:
        return(sorted([len(gap) for gap in gap_list], reverse=True)[0])
    else:
        return(0)

In [ ]:
# read and parse fasta files for each species
aligned_fasta = {}
for ortho in aligned_fasta_fn.keys():
    aligned_fasta[ortho] = {seq_record.id : seq_record 
                                      for seq_record in SeqIO.parse(aligned_fasta_fn[ortho],
                                                                    "fasta", alphabet=IUPAC.ambiguous_dna)}

In [ ]:
# parse coords from template species in aligned fasta's and trash entries w/ all gaps
coords = {} # coords[ortho][sp] = [coord, ]
for ortho in aligned_fasta:
    coords[ortho] = {}
    for sp in template_species_list:
        seq = str(aligned_fasta[ortho][sp].seq)
        temp_coords = findExonCoords(str(aligned_fasta[ortho][sp].seq))
        for start,end in temp_coords:
            cds = seq[start:end]
            if len(cds) != cds.count('-'):
                if sp not in coords[ortho]:
                    coords[ortho][sp] = (start,end)
                elif type(coords[ortho][sp]) is list:
                    coords[ortho][sp].append((start,end))
                else:
                    temp = coords[ortho][sp]
                    coords[ortho][sp] = [temp, (start,end)]

In [ ]:
# sanity check for multiple non gap template cds's per ortho,sp
for ortho in coords:
    for sp in coords[ortho]:
        if type(coords[ortho][sp]) is list:
            print("error, multiple non-gap template cds's for {},{}: {}".format(ortho, sp, coords[ortho][sp]))

In [ ]:
# Filter aligned exons
ortho_coords = {}
for ortho in coords:
    ortho_coords[ortho] = {}
    for sp in coords[ortho]:
        coord = coords[ortho][sp]

        # filter for length
        start, end = coord
        length = end - start
        if not min_cds_length <= length <= max_cds_length:
            continue

        # filter for gap percent
        seq = str(aligned_fasta[ortho][sp].seq[start:end])
        if gapPercent(seq) > max_gap_percent:
            continue

        # filter for gap length
        if longestGap(seq) > max_gap_length:
            continue

        # prep to filter for species membership of ortho
        if coord not in ortho_coords[ortho].keys():
            ortho_coords[ortho][coord] = set()
        ortho_coords[ortho][coord].add(sp)

# set of coords per ortho which were represented in all species
universal_ortho_coords = {}
for ortho in ortho_coords:
    for coord in ortho_coords[ortho]:
        sp_set = ortho_coords[ortho][coord]
        if len(sp_set) == len(template_species_list):
            if ortho not in universal_ortho_coords.keys():
                universal_ortho_coords[ortho] = set()
            universal_ortho_coords[ortho].add(coord)
        else:
            print("warning, {} {} has only {}".format(ortho, coord, sp_set))

In [ ]:
import pandas
from bokeh.charts import Histogram, show
from bokeh.io import output_notebook

In [ ]:
data = []
for ortho in sorted(universal_ortho_coords.keys()):
    for coord in sorted(universal_ortho_coords[ortho]):
        start, end = coord
        length = end - start
        data.append((ortho, coord, length))

df = pandas.DataFrame.from_records(data=data, columns=['ortho', 'coord', 'Aligned CDS Length'])
print(df.describe())

output_notebook()
hist = Histogram(df, values="Aligned CDS Length", title="Aligned CDS Length Histogram")
show(hist)

In [ ]:
# fasta prep
fasta_prep = {}
for ortho in universal_ortho_coords:
    fasta_prep[ortho] = []
    for coord in universal_ortho_coords[ortho]:
            temp_sp_list = []
            for sp in sorted(aligned_fasta[ortho]):
                start,end = coord
                seq = aligned_fasta[ortho][sp].seq[start:end]
                des = aligned_fasta[ortho][sp].description
                seqReq = SeqRecord(seq, id=sp, description=des)
                if sp in template_species_list:
                    fasta_prep[ortho].append(seqReq)
                else:
                    temp_sp_list.append(seqReq)

            fasta_prep[ortho].extend(temp_sp_list)


In [ ]:
pp(fasta_prep)

In [ ]:
len(fasta_prep)

In [ ]:
for ortho in fasta_prep:
    fasta_prep[ortho] = [seqReq for seqReq in fasta_prep[ortho] if (gapPercent(seqReq.seq) <= max_gap_percent) and (longestGap(seqReq.seq) <= max_gap_length)]
    
fasta_prep = {ortho:seq_list for ortho,seq_list in fasta_prep.items() if len(seq_list) >= 8}

In [ ]:
len(fasta_prep)

In [ ]:
# fasta output
for ortho in fasta_prep:
    with open(output_path + ortho + ".13spp.fasta", "w") as f:
        for seqReq in fasta_prep[ortho]:
            f.write(seqReq.format("fasta"))

In [ ]:
data = []
for ortho in sorted(fasta_prep.keys()):
    sp_count = len(fasta_prep[ortho])
    data.append((ortho, sp_count))

df = pandas.DataFrame.from_records(data=data, columns=['ortho', 'sp_count'])
print(df.describe())

output_notebook()
hist = Histogram(df, values="sp_count", title="Species Count per Ortho Histogram")
show(hist)

In [ ]:
# analize coverage
coverage = {}
for ortho,rec_list in fasta_prep.items():
    for seqRec in rec_list:
        sp = seqRec.id
        if sp not in coverage.keys():
            coverage[sp] = set()
        coverage[sp].add(ortho)

In [ ]:
for sp,count in sorted([(sp, len(coverage[sp])) for sp in coverage.keys()], key=lambda x: x[1]):
    print("species: {}\torthos: {}".format(sp,count))

In [ ]:
full_species_list = sorted(coverage.keys())
full_species_list

In [ ]:
species_index = {}
for i,sp in enumerate(full_species_list):
    species_index[sp] = i
species_index

In [ ]:
for sp1,sp2,count in sorted([(sp1, sp2, len(set.intersection(coverage[sp1], coverage[sp2]))) for sp1 in full_species_list for sp2 in full_species_list[species_index[sp1]:] if sp1 != sp2], key=lambda x: x[2]):
    print("sp1: {}\tsp2: {}\torthos: {}".format(sp1,sp2,count))

In [ ]:
set.intersection(coverage['Aobl'], coverage['Bjar'])

In [ ]:
data = []
for ortho in sorted(set.intersection(coverage['Aobl'], coverage['Bjar'])):
    sp_count = len(fasta_prep[ortho])
    data.append((ortho, sp_count))

df = pandas.DataFrame.from_records(data=data, columns=['ortho', 'sp_count'])
print(df.describe())

output_notebook()
hist = Histogram(df, values="sp_count", title="Histogram of Species Count per Ortho shaired by Aobl and Bjar")
show(hist)